In [1]:

from makiflow.layers import *
from makiflow.models.segmentation.segmentator import Segmentator
from makiflow.tf_scripts import set_main_gpu
set_main_gpu(1)
import tensorflow as tf
import numpy as np
import glob
import cv2
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt
import random
from makiflow.models.classificator import Classificator

from sklearn.utils import shuffle
from scipy.ndimage import gaussian_filter

In [2]:
import copy
def merging_masks(masks, index_of_main_mask, priority):
    """
    We choose the base mask which have index `index_of_main_mask` on which is put other classes
    according to `priority` of other classes

    Parameters
    ----------
    masks : list or numpy.array
        List or numpy array of masks.
    index_of_main_mask : int
        Index of base mask.
    priority : list
        List of priority of classes. From highest priority to least priority.
        Example: [9, 10, 3, 7], where 9 class have highest priority and will be put on others classes,
        on the other hand the 7th class have the lowest priority and will be put on the others the latest.
    Returns
    ---------
    main_mask : numpy.array
        The base mask after merging other classes.
    """

    # For easy access
    main_mask = copy.deepcopy(masks[index_of_main_mask])

    # Thanks to the `boolean_mask` higher priority class will not be erased
    boolean_mask = np.ones(main_mask.shape).astype(np.bool)

    for i in range(len(masks)):
        if i == index_of_main_mask:
            continue

        for prior in priority:
            temp_bool_mask = masks[i] == prior
            temp_bool_mask = temp_bool_mask * boolean_mask
            main_mask[temp_bool_mask] = prior
            boolean_mask[masks[i] == prior] = False
        
        print(boolean_mask[248,82])

    return main_mask


def mutate_masks(masks, mapping):
    """
    Remaps classes on the given `masks` according to the `mapping`.

    Parameters
    ----------
    masks : list or numpy.array
        List or numpy array of masks.
    mapping : list
        List of tuples: [(source_class_number, new_class_number)],
        where `source_class_number` will be changed to `new_class_number` in the `masks`.

    Returns
    ---------
    new_masks : the same type as `masks`
        New masks with changed class numbers.
    """
    if type(mapping) is not list or (len(mapping) != 0 and type(mapping[0]) is not tuple):
        raise TypeError('mapping should be list of typles')

    new_masks = copy.deepcopy(masks)

    for i in range(len(new_masks)):
        for elem in mapping:
            old_value = elem[0]
            new_value = elem[1]
            new_masks[i][masks[i] == old_value] = new_value

    return  new_masks

In [3]:
from makiflow.save_recover import Builder

In [4]:
import tensorflow as tf

In [79]:
def load_test_data(path):
    Xtrain, Ytrain = [], []
    masks = glob.glob(path + '/masks/*.bmp')
    masks.sort()
    for mask_name in tqdm(masks):
        img = cv2.imread(mask_name.replace('masks', 'images'))
        img = cv2.resize(img, (1024, 1024), interpolation=cv2.INTER_CUBIC)
        mask = cv2.imread(mask_name)
        mask = mask[:,:,0]
        mask = cv2.resize(mask, (1024, 1024), interpolation=cv2.INTER_NEAREST)
        Xtrain.append(img)
        Ytrain.append(mask)
        if np.max(mask) >= 10:
            print(np.max(mask), f' in image {mask_name} ')
    return Xtrain, Ytrain

In [102]:
Xtest, Ytest = load_test_data(path='/mnt/data/med_data/balanced_batches/batch_3/test_set')

100%|██████████| 16/16 [00:00<00:00, 163.50it/s]


In [81]:
Xtest = [i / 255 for i in Xtest]

In [82]:
#7

In [83]:
model = Builder.segmentator_from_json('/home/rustam/EyePit/Danil_tests/Train_separate_NN_test/Xception_1024_testUnet_standart_V6_binary.json', batch_size=16)

ses = tf.Session()
model.set_session(ses)

prefix_path = '/home/rustam/EyePit/Danil_tests/Train_separate_NN_test/'

model.load_weights(prefix_path + 'exp_2/7_class_only/exp_cv5_l2_v13/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=2/epoch_1/' + 'weights.ckpt')

answer_all = ses.run(tf.nn.softmax(model.predict(Xtest)))

ses.close()

answer_7 = np.argmax(answer_all,axis=3) * 6

Model is restored!
INFO:tensorflow:Restoring parameters from /home/rustam/EyePit/Danil_tests/Train_separate_NN_test/exp_2/7_class_only/exp_cv5_l2_v13/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=2/epoch_1/weights.ckpt
Weights are loaded.


In [84]:
#8

In [85]:
model = Builder.segmentator_from_json('/home/rustam/EyePit/Danil_tests/Train_separate_NN_test/Xception_1024_testUnet_standart_V6_binary.json', batch_size=16)

ses = tf.Session()
model.set_session(ses)

prefix_path = '/home/rustam/EyePit/Danil_tests/Train_separate_NN_test/'

model.load_weights(prefix_path + 'exp_3/only_8_class/exp_cv3/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=3/epoch_6/' +  'weights.ckpt')

answer_all = ses.run(tf.nn.softmax(model.predict(Xtest)))

ses.close()

answer_8 = np.argmax(answer_all,axis=3) * 7

Model is restored!
INFO:tensorflow:Restoring parameters from /home/rustam/EyePit/Danil_tests/Train_separate_NN_test/exp_3/only_8_class/exp_cv3/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=3/epoch_6/weights.ckpt
Weights are loaded.


In [86]:
#6

In [87]:
model = Builder.segmentator_from_json('/home/rustam/EyePit/Danil_tests/Train_separate_NN_test/Xception_1024_testUnet_standart_V6_binary.json', batch_size=16)

ses = tf.Session()
model.set_session(ses)

prefix_path = '/home/rustam/EyePit/Danil_tests/Train_separate_NN_test/'

model.load_weights(prefix_path + 'exp_4/only_6_class/exp_cv3/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=6/epoch_2/'+ 'weights.ckpt')

answer_all = ses.run(tf.nn.softmax(model.predict(Xtest)))

ses.close()

answer_6 = np.argmax(answer_all,axis=3) * 5

Model is restored!
INFO:tensorflow:Restoring parameters from /home/rustam/EyePit/Danil_tests/Train_separate_NN_test/exp_4/only_6_class/exp_cv3/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=6/epoch_2/weights.ckpt
Weights are loaded.


In [88]:
#others

In [89]:
model = Builder.segmentator_from_json('/home/rustam/EyePit/Danil_tests/Train_separate_NN_test/exp_4/other_classes/Xception_1024_testUnet_standart_V6.json', batch_size=16)

ses = tf.Session()
model.set_session(ses)

prefix_path = '/home/rustam/EyePit/Danil_tests/Train_separate_NN_test/exp_4/other_classes/'

model.load_weights(prefix_path + 'exp_cv3/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=8/epoch_12/' + 'weights.ckpt')

answer_all = ses.run(tf.nn.softmax(model.predict(Xtest)))

ses.close()

answer_others = np.argmax(answer_all,axis=3)

Model is restored!
INFO:tensorflow:Restoring parameters from /home/rustam/EyePit/Danil_tests/Train_separate_NN_test/exp_4/other_classes/exp_cv3/x-65/MakiSegmentator_gamma=2_opt_name=adam1_bsz=8/epoch_12/weights.ckpt
Weights are loaded.


In [93]:
np.max(answer_others[4])

5

In [94]:
answer_oth = mutate_masks(answer_others,mapping=[(1,1),(2,2),(3,3),(4,4),(5,8)])

In [ ]:
answer_merged = []
for i in range(len(Xtest)):
    answer_merged.append(merging_masks([answer_oth[i],answer_7[i], answer_8[i], answer_6[i]],0,priority=[7,6,5]))

In [96]:
len(answer_merged)

16

In [97]:
from makiflow.metrics.utils import one_hot
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

In [98]:
name_classes = [ str(i) for i in range(9)]

In [ ]:
answer_merged = np.asarray(answer_merged)

In [ ]:
answer_merged = answer_merged.reshape(-1)
answer_merged = one_hot(answer_merged, depth=9)
answer_merged = answer_merged.reshape(16, -1, 9)

In [103]:
Ytest = np.asarray(Ytest)

In [104]:
Ytest = mutate_masks(Ytest,mapping=[(4,0),(5,4),(6,5),(7,6),(8,7),(9,8)])

In [ ]:
EPSILON = 1e-9


def categorical_dice_coeff(P, L, use_argmax=False, ind_norm=True):
    """
    Calculates V-Dice for give predictions and labels.
    WARNING! THIS IMPLIES SEGMENTATION CONTEXT.
    Parameters
    ----------
    P : np.ndarray
        Predictions of a segmentator. Array of shape [batch_sz, W, H, num_classes].
    L : np.ndarray
        Labels for the segmentator. Array of shape [batch_sz, W, H]
    use_argmax : bool
        Converts the segmentator's predictions to one-hot format.
        Example: [0.4, 0.1, 0.5] -> [0., 0., 1.]
    ind_norm : bool
        Normalize each dice separately. Useful in case some classes don't appear
        on some images.
    """
    batch_sz = len(P)
    L = np.asarray(L)
    P = np.asarray(P)
    num_classes = P.shape[-1]
    if use_argmax:
        P = P.argmax(axis=3)
        P = P.reshape(-1)
        P = one_hot(P, depth=num_classes)
    P = P.reshape(batch_sz, -1, num_classes)
    L = L.reshape(batch_sz, -1)

    class_dices = np.zeros(num_classes)
    class_counts = np.zeros(num_classes) + EPSILON  # Smoothing to avoid division by zero
    for i in range(batch_sz):
        sample_actual = L[i]
        sample_pred = P[i]
        for j in range(num_classes):
            sub_actual = (sample_actual[:] == j).astype(np.int32)
            sub_confs = sample_pred[:, j]
            if np.sum(sub_actual) == 0 and np.sum(sub_confs) == 0:
                continue
            class_dices[j] += binary_dice(sub_confs, sub_actual)
            class_counts[j] += 1

    v_dice, dices = class_dices.mean() / batch_sz, class_dices / batch_sz
    if ind_norm:
        v_dice, dices = (class_dices / class_counts).mean(), class_dices / class_counts
    return v_dice, dices


def v_dice_coeff(P, L, use_argmax=False, one_hot_labels=False):
    """
    Calculates V-Dice for give predictions and labels.
    WARNING! THIS IMPLIES SEGMENTATION CONTEXT.
    Parameters
    ----------
    P : np.ndarray
        Predictions of a segmentator. Array of shape [batch_sz, W, H, num_classes].
    L : np.ndarray
        Labels for the segmentator. Array of shape [batch_sz, W, H]
    use_argmax : bool
        Converts the segmentator's predictions to one-hot format.
        Example: [0.4, 0.1, 0.5] -> [0., 0., 1.]
    one_hot_labels : bool
        Set to True if the labels (`L`) are already one-hot encoded, i.e. have the same
        shape as `P`.
    """
    # P has shape [batch_sz, W, H, num_classes]
    # L has shape [batch_sz, W, H]
    # RESHAPE TENSORS AND ONE-HOT LABELS
    # P -> [batch_sz, num_samples, num_classes]
    batch_sz = len(P)
    num_samples = P.shape[1] * P.shape[2]
    num_classes = P.shape[-1]
    if use_argmax:
        P = P.argmax(axis=3)
        P = P.reshape(batch_sz * num_samples)
        P = one_hot(P, depth=num_classes)
    P = P.reshape(batch_sz, num_samples, num_classes)

    if not one_hot_labels:
        # L -> [batch_sz*num_samples] -> [batch_sz*num_samples, num_classes] -> [batch_sz, num_samples, num_classes]
        L = L.reshape(batch_sz * num_samples)
        L = one_hot(L, depth=num_classes)
        L = L.reshape(batch_sz, num_samples, num_classes)

    # P has shape [batch_sz, num_samples, num_classes]
    # L has shape [batch_sz, num_samples, num_classes]
    R = P * L
    nums = R.sum(axis=1)

    P2 = P * P
    P2vec = P2.sum(axis=1)
    Lvec = L.sum(axis=1)
    dens = P2vec + Lvec
    dices_b = (2 * nums + EPSILON) / (dens + EPSILON)
    dices = dices_b.mean(axis=0)
    return dices.mean(), dices


def confusion_mat(
        p, l,
        use_argmax_p=False, use_argmax_l=False, to_flatten=False, normalize=[0, 1],
        save_path=None, dpi=150, annot=True
    ):
    """
    Creates confusion matrix for the given predictions `p` and labels `l`.
    The matrix consists of elements C(i,j). C(i,j) - number of samples of class i
    classified as class j.
    Parameters
    ----------
    p : np.ndarray
        Predictions.
    l : np.ndarray
        Corresponding labels.
    use_argmax_p : bool
        Set to true if prediction aren't sparse, i.e. `p` is an array of shape [..., num_classes].
    use_argmax_l : bool
        Set to True if labels aren't sparse (one-hot encoded), i.e. `l` is an array of shape [..., num_classes].
    to_flatten : bool
        Set to True if `p' and `l` are high-dimensional arrays.
    normalize : list 
        List of axes. The matrix will be normalized along these axes.
        Axis 1:
            Normalizing by the number of true samples per class.
            C(i,j) - percentage of samples of class i that classified as class j.
            Diagonal elements stand for recall.
        Axis 0:
            Normalizing by the number of the network predictions per class.
            C(i,j) - percentage of samples classified as j that actually belong to class i.
            Diagonal elements stand for precision.
        Leave the list empty if you want to get unnormalized matrix.
    save_path : str
        Saving path for the confusion matrix picture.
    dpi : int
        Affects the size of the saved confusion matrix picture.
    annot : bool
        Set to true if you want to see actual numbers (classes) on the matrix picture.
    
    Returns
    -------
    list
        Confusion matrices.
    """
    if use_argmax_p:
        p = p.argmax(axis=-1)

    if use_argmax_l:
        l = l.argmax(axis=-1)

    if to_flatten:
        p = p.reshape(-1)
        l = l.reshape(-1)

    mat = np.asarray(confusion_matrix(l, p), dtype=np.float32)
    del p
    del l
    
    assert(len(normalize) < 3)
    
    if len(normalize) == 2:
        
        mats = []
        for ax in normalize:
            # Normalizing along axis 0
            if ax == 0:
                temp_mat = mat / mat.sum(axis=0)
            # Normalizing along axis 1
            elif ax == 1:
                temp_mat = (mat.T / mat.sum(axis=1)).T
            else:
                raise RuntimeError(f"Unknown axis: {ax}")
            temp_mat = np.round(temp_mat, decimals=2)
            mats += [temp_mat]
            
        if save_path is not None:
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6.4))
            sns.heatmap(mats[0], annot=annot, ax=axes[0])
            axes[0].set_title(f'Axis {normalize[0]}')
            sns.heatmap(mats[1], annot=annot, ax=axes[1])
            axes[1].set_title(f'Axis {normalize[1]}')
            fig.savefig(save_path)
            plt.close(fig)
        
        return mats

    if len(normalize) == 1:
        ax = normalize[0]
        # Normalizing along axis 0
        if ax == 0:
            mat = mat / mat.sum(axis=0)
        # Normalizing along axis 1
        elif ax == 1:
            mat = (mat.T / mat.sum(axis=1)).T
        else:
            raise RuntimeError(f"Unknown axis: {ax}")
        mat = np.round(mat, decimals=2)

    if save_path is not None:
        conf_mat = sns.heatmap(mat, annot=annot)
        conf_mat.figure.savefig(save_path, dpi=dpi)
        plt.close(conf_mat.figure)

    return [mat]


def binary_dice(predicted, actual):
    num = np.sum(predicted * actual)
    den = np.sum(predicted * predicted) + np.sum(actual)
    return (2 * num + EPSILON) / (den + EPSILON)

In [107]:
# COMPUTE DICE AND CREATE CONFUSION MATRIX
v_dice_val, dices = categorical_dice_coeff(answer_merged, Ytest, use_argmax=False)

print('V-Dice:', v_dice_val)
for i, class_name in enumerate(name_classes):
    print(f'{class_name}:', dices[i])

# Compute and save matrix
conf_mat_path = f'mat_epoch.png'
print('Computing confusion matrix...')
confusion_mat(
    answer_merged, Ytest, use_argmax_p=True, to_flatten=True,
    save_path=conf_mat_path, dpi=175
)

V-Dice: 0.5202994572386749
0: 0.9579113947899555
1: 0.8794554997769812
2: 0.681739886450964
3: 0.7277670772843542
4: 0.5502047641834295
5: 0.1876329254669759
6: 0.04087696833793567
7: 0.030878885998157603
8: 0.6262277128593211
Computing confusion matrix...


[array([[0.95, 0.04, 0.26, 0.23, 0.3 , 0.34, 0.62, 0.58, 0.24],
        [0.  , 0.95, 0.  , 0.01, 0.  , 0.05, 0.01, 0.  , 0.  ],
        [0.  , 0.  , 0.72, 0.  , 0.04, 0.  , 0.  , 0.  , 0.  ],
        [0.03, 0.01, 0.  , 0.75, 0.01, 0.03, 0.01, 0.03, 0.02],
        [0.  , 0.  , 0.01, 0.  , 0.62, 0.01, 0.08, 0.03, 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.57, 0.01, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.27, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.34, 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.74]],
       dtype=float32),
 array([[0.97, 0.  , 0.  , 0.03, 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.11, 0.86, 0.  , 0.03, 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.31, 0.  , 0.67, 0.  , 0.02, 0.  , 0.  , 0.  , 0.  ],
        [0.29, 0.  , 0.  , 0.7 , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.3 , 0.  , 0.03, 0.01, 0.65, 0.  , 0.02, 0.  , 0.  ],
        [0.73, 0.  , 0.  , 0.01, 0.  , 0.25, 0.  , 0.  , 0.01],
        [0.64, 0

In [37]:
import copy

In [38]:
test = copy.deepcopy(answer_all)

In [39]:
test = test.reshape(-1)
test = one_hot(test, depth=10)
test = test.reshape(16, -1, 10)

In [40]:
# COMPUTE DICE AND CREATE CONFUSION MATRIX
v_dice_val, dices = categorical_dice_coeff(test, Ytest, use_argmax=False)

print('V-Dice:', v_dice_val)
for i, class_name in enumerate(name_classes):
    print(f'{class_name}:', dices[i])

# Compute and save matrix
conf_mat_path = f'mat_epoch.png'
print('Computing confusion matrix...')
confusion_mat(
    test, Ytest, use_argmax_p=True, to_flatten=True,
    save_path=conf_mat_path, dpi=175
)

V-Dice: 0.48050322294056286
0: 0.9563937567568369
1: 0.8662955174956187
2: 0.6821455532621923
3: 0.7224625801151546
4: 5.416421907765949e-13
5: 0.5312853465411852
6: 0.3106011384624071
7: 0.052382682113361716
8: 0.06867535819430415
9: 0.6147902964640262
Computing confusion matrix...


/home/rustam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:167: RuntimeWarning: invalid value encountered in true_divide


[array([[0.95, 0.11, 0.25, 0.23,  nan, 0.41, 0.25, 0.3 , 0.59, 0.3 ],
        [0.  , 0.85, 0.  , 0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.74, 0.  ,  nan, 0.05, 0.  , 0.  , 0.  , 0.  ],
        [0.03, 0.03, 0.  , 0.76,  nan, 0.01, 0.01, 0.  , 0.01, 0.02],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.49, 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.74, 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.03, 0.  , 0.69, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.  , 0.01, 0.39, 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.67]],
       dtype=float32),
 array([[0.96, 0.  , 0.  , 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.06, 0.93, 0.  , 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.31, 0.  , 0.65, 0.  , 0.  , 0.04, 0.  , 0.  , 0.  , 0.  ],
        [0.3 , 0.01, 0.  , 0.69, 0.  , 0.  , 0.  , 0.  , 0.  , 0. 

In [69]:
len(test_2)

16

In [70]:
test_2[0].shape

(1024, 1024)

In [71]:
test_2 = np.asarray(test_2)

In [72]:
test_2 = test_2.reshape(-1)
test_2 = one_hot(test_2, depth=10)
test_2 = test_2.reshape(16, -1, 10)

In [73]:
# COMPUTE DICE AND CREATE CONFUSION MATRIX
v_dice_val, dices = categorical_dice_coeff(test_2, Ytest, use_argmax=False)

print('V-Dice:', v_dice_val)
for i, class_name in enumerate(name_classes):
    print(f'{class_name}:', dices[i])

# Compute and save matrix
conf_mat_path = f'mat_epoch.png'
print('Computing confusion matrix...')
confusion_mat(
    test_2, Ytest, use_argmax_p=True, to_flatten=True,
    save_path=conf_mat_path, dpi=175
)

V-Dice: 0.48050322294056286
0: 0.9563937567568369
1: 0.8662955174956187
2: 0.6821455532621923
3: 0.7224625801151546
4: 5.416421907765949e-13
5: 0.5312853465411852
6: 0.3106011384624071
7: 0.052382682113361716
8: 0.06867535819430415
9: 0.6147902964640262
Computing confusion matrix...


/home/rustam/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:167: RuntimeWarning: invalid value encountered in true_divide


[array([[0.95, 0.11, 0.25, 0.23,  nan, 0.41, 0.25, 0.3 , 0.59, 0.3 ],
        [0.  , 0.85, 0.  , 0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.74, 0.  ,  nan, 0.05, 0.  , 0.  , 0.  , 0.  ],
        [0.03, 0.03, 0.  , 0.76,  nan, 0.01, 0.01, 0.  , 0.01, 0.02],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.49, 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.74, 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.03, 0.  , 0.69, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.  , 0.01, 0.39, 0.  ],
        [0.  , 0.  , 0.  , 0.  ,  nan, 0.  , 0.  , 0.  , 0.  , 0.67]],
       dtype=float32),
 array([[0.96, 0.  , 0.  , 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.06, 0.93, 0.  , 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.31, 0.  , 0.65, 0.  , 0.  , 0.04, 0.  , 0.  , 0.  , 0.  ],
        [0.3 , 0.01, 0.  , 0.69, 0.  , 0.  , 0.  , 0.  , 0.  , 0. 